# 네이버 영화평 감성분석
- Tokenizer 함수
- TfidfVectorizer + LogisticRegression

In [1]:
!pip install Konlpy > /dev/null

In [2]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [3]:
from google.colab import files
up = files.upload()

Saving naver_movie_test_전처리완료.tsv to naver_movie_test_전처리완료.tsv
Saving naver_movie_train_전처리완료.tsv to naver_movie_train_전처리완료.tsv


In [7]:
list(up.keys())[1]

'naver_movie_train_전처리완료.tsv'

In [8]:
train_df = pd.read_csv(list(up.keys())[1], sep='\t')
test_df = pd.read_csv(list(up.keys())[0], sep='\t')
train_df.shape, test_df.shape

((145393, 3), (48852, 3))

In [14]:
train_df.head(3)

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


- Tokenizer 함수 정의

In [9]:
from konlpy.tag import Okt
okt = Okt()

In [10]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','을','ㅋㅋ','ㅠㅠ','ㅎㅎ']

In [11]:
def okt_tokenizer(text):
    morphs = okt.morphs(text, stem=True)
    tokens = [word for word in morphs if word not in stopwords]
    return tokens

In [12]:
okt_tokenizer('열심히 일한 당신 주말엔 여행을 떠나봐요.')

['열심히', '일', '당신', '주말', '엔', '여행', '떠나다', '보다', '.']

- Pipeline으로 피쳐 변환과 분류를 동시에

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [15]:
pipeline = Pipeline([
    ('TFIDF', TfidfVectorizer(tokenizer=okt_tokenizer)),
    ('LR', LogisticRegression(random_state=2022))
])
%time pipeline.fit(train_df.document, train_df.label)

CPU times: user 3min 50s, sys: 5.01 s, total: 3min 55s
Wall time: 3min 45s


Pipeline(steps=[('TFIDF',
                 TfidfVectorizer(tokenizer=<function okt_tokenizer at 0x7f1165ec0830>)),
                ('LR', LogisticRegression(random_state=2022))])

In [16]:
pipeline.score(test_df.document, test_df.label)

0.8425038892982887

- 실제 데이터 테스트

In [30]:
import re
reviews = ['모든 국민이 봤으면 하는 영화입니다.',
           '생각보다 지루하고 별로였네요... 보면서 좀 졸았습니다.']
reviews = map(lambda x: re.sub('[^가-힣]',' ',x), reviews)

In [31]:
pipeline.predict(reviews)

array([1, 0])

- 최적 파라메터 찾기
    - 매 시행시마다 한글 형태소 분석을 하느라 시간이 너무 오래 걸림
    - 최적 파라메터를 찾으려고 하면 한글 형태소 분석을 한 데이터로 할 것

In [36]:
from sklearn.model_selection import GridSearchCV
params = {
    'TFIDF__ngram_range': [(1,1),(1,2)],
    'TFIDF__max_df': [0.95, 0.98],
    'LR__C': [1, 5]
}

In [41]:
grid_pipe = GridSearchCV(pipeline, params, scoring='accuracy', cv=3)
# %time grid_pipe.fit(train_df.document, train_df.label)

- CountVectorizer 사례에서 찾은 최적의 파라메터로 평가

In [39]:
pipeline = Pipeline([
    ('TFIDF', TfidfVectorizer(tokenizer=okt_tokenizer, max_df=0.95, ngram_range=(1,2))),
    ('LR', LogisticRegression(random_state=2022))
])
%time pipeline.fit(train_df.document, train_df.label)

CPU times: user 6min 6s, sys: 15.4 s, total: 6min 22s
Wall time: 6min 3s


Pipeline(steps=[('TFIDF',
                 TfidfVectorizer(max_df=0.95, ngram_range=(1, 2),
                                 tokenizer=<function okt_tokenizer at 0x7f1165ec0830>)),
                ('LR', LogisticRegression(random_state=2022))])

In [40]:
pipeline.score(test_df.document, test_df.label)

0.8583067223450421